In [2]:
import os.path

import pandas as pd


%run 0-0-common.ipynb

In [4]:
%%time

from shared_code.utility.storage.azure_file_storage import AzureFileStorageAdapter
from adlfs import AzureBlobFileSystem

storage_adapter: AzureFileStorageAdapter = AzureFileStorageAdapter('data')
file_system: AzureBlobFileSystem = storage_adapter.get_file_storage()


CPU times: total: 0 ns
Wall time: 0 ns


In [5]:
%%time

extant_raw = pd.read_parquet('data/processed_raw_data.parquet', filesystem=file_system, engine='pyarrow', schema=schema)

print("=== RAW DATA ===")
display(extant_raw.shape)

extant_raw.drop_duplicates(subset=['id'], keep='first', inplace=True)

print("=== Deduplicated DATA ===")
display(extant_raw.shape)

=== RAW DATA ===


(41505, 15)

=== Deduplicated DATA ===


(38051, 15)

CPU times: total: 703 ms
Wall time: 2.14 s


In [6]:
%%time

extant_raw_indexed = extant_raw.set_index('id')

extant_raw_indexed['id'] = extant_raw.index

print(extant_raw_indexed.shape)
display(extant_raw_indexed)

(38051, 15)


,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags,id
id,,,,,,,,,,,,,,,
128z5ns,celebrities,A-Sexy-Name,Rachel Weisz,cars are parked on the side of the road in the...,f09b1c0c5dea50c934081a04c83d4d33,/r/celebrities/comments/128z5ns/rachel_weisz/,https://i.redd.it/nv80afddacra1.jpg,f09b1c0c5dea50c934081a04c83d4d33.jpg,D:\data\images\celebrities\f09b1c0c5dea50c9340...,None,True,False,None,None,0
11yk8db,prettyasiangirls,MisoShiru520,Library,blonde woman with blonde hair and tattoos on h...,9d5b7331b941ecf3fdbc8ad322303825,/r/prettyasiangirls/comments/11yk8db/library/,https://i.redd.it/ht4d2wuusapa1.jpg,9d5b7331b941ecf3fdbc8ad322303825.jpg,D:\data\images\prettyasiangirls\9d5b7331b941ec...,None,True,False,None,None,1
11hfioi,KoreanHotties,Majestic_Painter8660,Underboob bikini,a man with a beard and a beard sitting in fron...,a154a56b1b87cf3a0bc93600c86e4585,/r/KoreanHotties/comments/11hfioi/underboob_bi...,https://i.redd.it/cimr17q84jla1.jpg,a154a56b1b87cf3a0bc93600c86e4585.jpg,D:\ImageDoesNotExist.png,None,False,False,None,None,2
1000mjs,spaceporn,MorningStar_imangi,Northern Lights above Lofoten,a view of a view of a large green and purple a...,2c39ce1290fba541abd0b004b09da6b2,/r/spaceporn/comments/1000mjs/northern_lights_...,https://i.redd.it/7s5aafaqkb9a1.jpg,7s5aafaqkb9a1.jpg,D:\data\images\spaceporn\7s5aafaqkb9a1.jpg,None,True,False,None,None,3
1000qpd,spaceporn,MorningStar_imangi,Viking Lights,a scene of a boat is sitting on the shore of a...,0f72de47c69ff50eca5fa3990215f4ac,/r/spaceporn/comments/1000qpd/viking_lights/,https://i.redd.it/abojw7lqlb9a1.jpg,abojw7lqlb9a1.jpg,D:\data\images\spaceporn\abojw7lqlb9a1.jpg,None,True,False,None,None,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131dso7,DLAH,Wallydinger123,Happy B Day,there are three women posing for a picture in ...,ebb4b4a88dc79e2f2f57b10f32bf03c0,/r/DLAH/comments/131dso7/happy_b_day/,https://i.redd.it/36gurz533jwa1.jpg,ebb4b4a88dc79e2f2f57b10f32bf03c0.jpg,D:\data\images\DLAH\ebb4b4a88dc79e2f2f57b10f32...,None,True,False,None,None,41500
131eznp,DLAH,Long_dong111,Perfect,araffe woman in a white dress sitting on a pur...,5c944c6d62b1ae182c5a6a40c64195f4,/r/DLAH/comments/131eznp/perfect/,https://i.redd.it/dnbbx4zucjwa1.jpg,5c944c6d62b1ae182c5a6a40c64195f4.jpg,D:\data\images\DLAH\5c944c6d62b1ae182c5a6a40c6...,None,True,False,None,None,41501
131g3am,DLAH,ThrownAwayMiles,DLAH left or right,two women in short dresses posing for a pictur...,ab1f8fc33c85f4cfce12c810cf22229e,/r/DLAH/comments/131g3am/dlah_left_or_right/,https://i.redd.it/zpp70oor3lwa1.jpg,ab1f8fc33c85f4cfce12c810cf22229e.jpg,D:\data\images\DLAH\ab1f8fc33c85f4cfce12c810cf...,None,True,False,None,None,41502


CPU times: total: 78.1 ms
Wall time: 79 ms


In [1]:
cloud_ready_files = [os.path.basename(file) for file in file_system.ls('data/image/')]
display(cloud_ready_files)

NameError: name 'file_system' is not defined

In [7]:
curated = pd.read_parquet('data/parquet/back.parquet', filesystem=file_system, engine='pyarrow', schema=schema)
already_curated_ids = curated.loc[curated['accept'] == True].id.to_list()
display(already_curated_ids)

['1013bdt',
 '105mekt',
 '105qvgl',
 '105rpcj',
 '105styc',
 '105tw6t',
 '106673i',
 '106lsw4',
 '106mh03',
 '10ce95f',
 '10cywc7',
 '10i1flv',
 '10i1qkq',
 '10i56sz',
 '10i8he7',
 '10iuhd0',
 '10nyadr',
 '10o96va',
 '10odumt',
 '10ok9gc',
 '10olva7',
 '10onw06',
 '10osnnk',
 '10pl8zt',
 '10pq54f',
 '10prtpi',
 '10q17m7',
 '10q1t59',
 '10q46tx',
 '10q6wg2',
 '10q8gj9',
 '10q97um',
 '10qdfml',
 '10qizmn',
 'ymvhze',
 'yn2irc',
 'ynlbt6',
 'yo71q9',
 'ysgkjg',
 'ytjs2s',
 'ytttbp',
 'yugsit',
 'ywiv0k',
 'yxhlpc',
 'yxt6t9',
 'z4rzpq',
 'z4u1xz',
 'z6gu0e',
 'za5a21',
 'zawc2b',
 'zbn64p',
 'zbvyyg',
 'zitlo7',
 'znqjpa',
 'zp68ea',
 'zp99sa',
 'zyscw8',
 '1019kyo',
 '1032j8y',
 '103h4hd',
 '106hchf',
 '108e7ml',
 '109n66u',
 '109x870',
 '10bpm50',
 '10f78ow',
 '10i05he',
 '10qddse',
 'ymlcfe',
 'z7fn51',
 'ze9wml',
 'zqyw7y',
 'zrkhhb',
 '1027i9a',
 '103zqoc',
 '10gzlqt',
 '1213x1h',
 '129z5l8']

In [16]:
from tqdm import tqdm

records = extant_raw_indexed.to_dict(orient='records')
curated_records = curated.to_dict(orient='records')

extant_records = []

for record in tqdm(records, total=len(records), desc='Checking Local records'):
    try:
        if os.path.exists(record['path']):
            extant_records.append(record)
    except Exception as e:
        continue

out = []

# for record in tqdm(extant_records, total=len(extant_records), desc='Saving Extant Records'):
#     link = record['permalink'].split('/')
#     record['id'] = link[4]
#
#     if record['id'] in already_curated_ids:
#         continue
#     if not record['id'] + ".jpg" in cloud_ready_files:
#         print(record['path'])
#         file_system.upload(record['path'], f'data/image/{record["id"]}.jpg', overwrite=True)
#     else:
#         record['path'] = f'data/image/{record["id"]}.jpg'
#         record['image_name'] = f'{record["id"]}.jpg'
#         record['accept'] = False
#         record['curated'] = False
#         record['tags'] = []
#         try:
#             record['model'] = [source['name'] for source in sources if record['subreddit'] in source['data']][0]
#             record['tags'] = []
#         except:
#             pass
#         if record['id'] in [curated_record['id'] for curated_record in curated_records]:
#             record['model'] = ""
#             record['tags'] = []
#             continue
#         out.append(record)
#
# new_things = pd.DataFrame(data=out)

Saving Extant Records: 100%|██████████| 33710/33710 [10:58<00:00, 51.18it/s] 


In [17]:
# display(new_things)

""


In [10]:
# foo.to_parquet('data/parquet/', filesystem=file_system, engine='pyarrow', schema=schema, partition_cols=['subreddit'], index=['id'])

In [1]:
import pandas

foo = pandas.read_parquet('data/parquet/back.parquet', filesystem=file_system, engine='pyarrow', schema=schema)

# foo.set_index('id', inplace=True, drop=False)

# foo.drop_duplicates(subset=['id'], keep='last', inplace=True)

print(foo.shape)

display(foo)

# foo.to_parquet('data/parquet/back.parquet', filesystem=file_system, engine='pyarrow', schema=schema)
# df_index.to_parquet('data/parquet/', filesystem=file_system, engine='pyarrow', schema=schema)
# records = foo.to_dict(orient='records')
#
# thing = records[0]
#
# bar = pandas.DataFrame(data=[thing], index=['id'])
#
# display(bar)
#
# foo.update(bar, overwrite=True)
#
# print(foo.shape)
#
# display(foo)
#
# display(foo.loc[foo['id'] == '106mh03'])

NameError: name 'file_system' is not defined

In [12]:
# temp = foo
# display(temp)

In [13]:
# temp = foo
# display(temp)
# df_index.to_parquet('data/parquet/ready_to_curate.parquet', filesystem=file_system, engine='pyarrow', schema=schema)

In [14]:

# foo = pandas.DataFrame(data=[{
#     'id': '10kzbey',
#     'path': 'data/image/1.jpg',
#     'image_name': '1.jpg',
#     'accept': True,
#     'curated': True,
#     'model': 'test',
#     'tags': []
# }])
#
# temp = df

# display(df.loc[df['id'] == '105styc'])

# display(temp.loc[temp['id'] == '10kzbey'])

# display(temp.loc[temp['id'] == '10kzbey'])